In [ ]:
import random
import numpy as np
num_states = 100
num_actions = 13  # Number of possible actions (s, b, r)
q_table = np.zeros((num_states,num_actions))
state_index_map = {}
max_epsilon = 1.0
min_epsilon = 0.1  # Minimum epsilon value
decay_rate = 0.005  # Rate at which epsilon decreases
print(q_table)
class Player:
    def __init__(self):
        self.reset()

    def reset(self):
        self.bullets = 0
        self.is_alive = True
        self.current_block = None
        self.current_shoot = None
        self.shooter = {1: "Gun", 2: "Double Gun", 3: "Headshot", 4: "Bazooka", 5: "RasenGun", 6: "Double Bazooka"}
        self.blocker = {1: "Shield", 2: "Double Shield", 3: "Head Shield", 4: "Barricade", 5: "RasenGun Catcher", 6: "Epic Barricade"}
        self.shooted = False
        self.blocked = False
        self.charged = False
        self.opponent_bullets=0

    def set_bullets(self, num_bullets):
        self.bullets = num_bullets
    def set_opponent_bullets(self, opp_bullets):
        self.opponent_bullets = opp_bullets

    def set_alive(self, alive_status):
        self.is_alive = alive_status

    def check_alive(self):
        return self.is_alive

    def set_shoot(self, shoot_option):
        self.current_shoot = shoot_option
        self.shooted = True

    def get_shoot(self):
        return self.current_shoot

    def set_block(self, block_option):
        self.current_block = block_option
        self.blocked = True

    def set_charged(self, charge):
        self.charged=charge

    def get_block(self):
        return self.current_block

    def get_bullets(self):
        return self.bullets

    def get_opponent_bullets(self):
        return self.opponent_bullets

    def is_alive(self):
        return self.is_alive

    def has_shooted(self):
        return self.shooted

    def has_blocked(self):
        return self.blocked

    def has_charged(self):
        return self.charged

def is_death(me_shoot, opp_shoot, me_block,  opp_block,):
    death = False

    if me_shoot>0 and opp_shoot>0:
      if me_shoot<opp_shoot:
        death=True
    if me_shoot==0 and opp_shoot>0:
      if me_block!=opp_shoot:
        death=True

    return death
def get_state_index(state):
    if state not in state_index_map:
        state_index_map[state] = len(state_index_map)  # Assign a new index to unseen states
    return state_index_map[state]

def ai_choice(own,opponent):
      if own ==0 and opponent==0:
            return "r", None
      if own ==0:
        choice = random.choice(['b', 'r'])
        if choice == 'b':
          much = random.randint(1,min( 6, opponent))
          return choice, much
        else:
            return choice, None
      elif opponent==0:
        choice = random.choice(['s', 'r'])
        if choice == 's' and own >= 1:
          much = random.randint(1,min( 6,own))
          return choice, much
        else:
            return choice, None
      else:
        choice = random.choice(['s', 'b', 'r'])
        if choice == 's' and own >= 1:
            much = random.randint(1,min( 6, own))
            return choice, much

        elif choice == 'b'and opponent >= 1:
            much = random.randint(1, min( 6,own))
            return choice, much
        else:
            return choice, None
def pick_index_with_probability(arr):
    arr=np.square(arr)
    # Calculate cumulative probabilities
    total_prob = sum(arr)
    cumulative_probs = [sum(arr[:i+1]) / total_prob for i in range(len(arr))]

    # Generate a random number
    random_num = random.random()

    # Find the chosen index
    chosen_index = None
    for i, cum_prob in enumerate(cumulative_probs):
        if random_num <= cum_prob:
            chosen_index = i
            break

    return chosen_index

def ai_choice_q_table(own, opponent,iteration):
    state = (own, opponent)
    state = get_state_index(state)
    choice_logic=False
    current_epsilon = max(min_epsilon, max_epsilon * np.exp(-decay_rate * iteration))
    if np.random.uniform(0, 1) < current_epsilon:
      if own ==0 and opponent==0:
            return "r", None,state
      if own ==0:
        choice = random.choice(['b', 'r'])
        if choice == 'b':
          much = random.randint(1,min( 6, opponent))
          return choice, much,state
        else:
            return choice, None,state
      elif opponent==0:
        choice = random.choice(['s', 'r'])
        if choice == 's' and own >= 1:
          much = random.randint(1,min( 6,own))
          return choice, much,state
        else:
            return choice, None,state
      else:
        choice = random.choice(['s', 'b', 'r'])
        if choice == 's' and own >= 1:
            much = random.randint(1,min( 6, own))
            return choice, much,state

        elif choice == 'b'and opponent >= 1:
            much = random.randint(1, min( 6,own))
            return choice, much,state
        else:
            return choice, None,state
    else:
        while choice_logic==False:
          action_values = q_table[state]
          print(action_values)
          pick=pick_index_with_probability((np.array(action_values))+0.00001)
          choice = ['r','s1','s2','s3','s4','s5','s6', 'b1','b2','b3','b4','b5','b6'][pick]
          if len(choice)>1:
            much=int(choice[1])
            choice=choice[0]
          else:
            much=None
          if choice=="s" and own>=much:
            choice_logic=True
          if choice=="r":
            choice_logic=True
          if choice=="b":
            choice_logic=True
        return choice, much ,state


def update_q_value(q_table, state_index, action, reward, next_state, alpha, gamma):
    current_value = q_table[state_index][action]
    max_next_value = np.max(q_table[next_state])
    new_value = (1 - alpha) * current_value + alpha * (reward + gamma * max_next_value)
    q_table[state_index][action] = new_value

def main():
    player1wins = 0
    pcwins = 0

    for i in range(10000):
        player1 = Player()
        pc = Player()
        turn_count = 1

        while player1.is_alive and pc.is_alive:
            logic = False

            if turn_count == 1:
                print("Let's watch two AIs play against each other!")
                print("\nTurn 1!")
                print("Both of them charged.\n")
                player1.set_bullets(1)
                pc.set_bullets(1)
            else:
                while not logic:
                    print(f"Player 1 bullets: {player1.get_bullets()}")
                    print(f"PC bullets: {pc.get_bullets()}\n")
                    player1.set_opponent_bullets(pc.get_bullets())
                    pc.set_opponent_bullets(player1.get_bullets())
                    choice, amount ,state= ai_choice_q_table( player1.get_bullets(),player1.get_opponent_bullets(),i)
                    if choice == 's':
                        player1.set_shoot(amount)
                        player1.set_bullets(player1.get_bullets() - amount)
                        player1.set_charged(False)
                        print(f"Player 1 pulled out a {player1.get_shoot()} and", end=" ")
                        reward = amount*0.75
                        next_state = (player1.get_bullets(), player1.get_opponent_bullets())
                        update_q_value(q_table, state, ['r','s1','s2','s3','s4','s5','s6', 'b1','b2','b3','b4','b5','b6'].index(choice + str(amount) if amount is not None else choice), reward, next_state, alpha=0.1, gamma=0.9)
                    elif choice == 'b':
                        player1.set_block(amount)
                        player1.set_shoot(0)
                        player1.set_charged(False)
                        print(f"Player 1 blocked with a {player1.get_block()} and", end=" ")
                    elif choice == 'r':
                        player1.set_bullets(player1.get_bullets() + 1)
                        player1.set_shoot(0)
                        player1.set_block(0)
                        player1.set_charged(True)
                        reward = 0.6
                        next_state = (player1.get_bullets(), player1.get_opponent_bullets())
                        update_q_value(q_table, state, ['r','s1','s2','s3','s4','s5','s6', 'b1','b2','b3','b4','b5','b6'].index(choice + str(amount) if amount is not None else choice), reward, next_state, alpha=0.1, gamma=0.9)
                        print("Player 1 charged and", end=" ")

                    choice, amount, state = ai_choice_q_table(pc.get_bullets(), pc.get_opponent_bullets(), i)
                    if choice == 's':
                        pc.set_shoot(amount)
                        pc.set_bullets(pc.get_bullets() - amount)
                        pc.set_charged(False)
                        print(f"the CPU pulled out a {pc.get_shoot()}!")
                        pc.shooted = True
                    elif choice == 'b':
                        pc.set_block(amount)
                        pc.set_shoot(0)
                        pc.set_charged(False)
                        print(f"the CPU blocked with a {pc.get_block()}!")
                        pc.blocked = True
                    elif choice == 'r':
                        pc.set_bullets(pc.get_bullets() + 1)
                        pc.set_shoot(0)
                        pc.set_block(0)
                        pc.set_charged(True)
                        pc.set_opponent_bullets(player1.get_bullets())
                        print("the CPU charged.")

                    logic = True

                if pc.get_shoot() == player1.get_block():
                    reward = 0.3 if player1.is_alive else 0
                    next_state = (pc.get_bullets(), pc.get_opponent_bullets())
                    update_q_value(q_table, state, ['r','s1','s2','s3','s4','s5','s6', 'b1','b2','b3','b4','b5','b6'].index(choice + str(amount) if amount is not None else choice), reward, next_state, alpha=0.1, gamma=0.9)

                if is_death(player1.get_shoot(), pc.get_shoot(), player1.get_block(), pc.get_block()):
                    player1.set_alive(False)

                if is_death(pc.get_shoot(), player1.get_shoot(), pc.get_block(), player1.get_block()):
                    pc.set_alive(False)

                reward = 3 if pc.is_alive else 0
                reward = 3 if player1.is_alive else 0
                next_state = (pc.get_bullets(), pc.get_opponent_bullets())
                update_q_value(q_table, state, ['r','s1','s2','s3','s4','s5','s6', 'b1','b2','b3','b4','b5','b6'].index(choice + str(amount) if amount is not None else choice), reward, next_state, alpha=0.1, gamma=0.9)
            if not player1.is_alive:
                print("PC AI won!")
                pcwins = pcwins + 1
                break
            elif not pc.is_alive:
                print("Player 1 won!")
                player1wins = player1wins + 1
                break
            turn_count += 1

    print("Player 1 wins:", player1wins)
    print("PC AI wins:", pcwins)

if __name__ == "__main__":
    main()


Streaming output truncated to the last 5000 lines.

[ 2.70344701 17.55480605  0.          0.          0.          0.
  0.          2.98610789  0.          0.          0.          0.
  0.        ]
Player 1 pulled out a 1 and [ 2.70344701 17.45425799  0.          0.          0.          0.
  0.          2.98610789  0.          0.          0.          0.
  0.        ]
the CPU pulled out a 1!
Player 1 bullets: 0
PC bullets: 0

[4.44311126 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.        ]
Player 1 charged and [4.81200363 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.        ]
the CPU charged.
Player 1 bullets: 1
PC bullets: 1

[ 2.70344701 17.57971541  0.          0.          0.          0.
  0.          2.98610789  0.          0.          0.          0.
  0.        ]
Player 1 pulled out a 1 and [ 2.70344701 17.47891826  0.          0.          0.          0.
 

In [ ]:
def ai_choice_q_table(own, opponent):
    state = (own, opponent)
    state = get_state_index(state)
    choice_logic=False
    while choice_logic==False:
          action_values = q_table[state]
          print(action_values)
          pick=pick_index_with_probability((np.array(action_values))+0.0001)
          choice = ['r','s1','s2','s3','s4','s5','s6', 'b1','b2','b3','b4','b5','b6'][pick]
          if len(choice)>1:
            much=int(choice[1])
            choice=choice[0]
          else:
            much=None
          if choice=="s" and own>=much:
            choice_logic=True
          if choice=="r":
            choice_logic=True
          if choice=="b":
            choice_logic=True
    return choice, much ,state


def main():
    player1wins = 0
    pcwins = 0

    for i in range(1000):
        player1 = Player()
        pc = Player()
        turn_count = 1

        while player1.is_alive and pc.is_alive:
            logic = False

            if turn_count == 1:
                print("Let's watch two AIs play against each other!")
                print("\nTurn 1!")
                print("Both of them charged.\n")
                player1.set_bullets(1)
                pc.set_bullets(1)
            else:
                while not logic:
                    print(f"Player 1 bullets: {player1.get_bullets()}")
                    print(f"PC bullets: {pc.get_bullets()}\n")
                    player1.set_opponent_bullets(pc.get_bullets())
                    pc.set_opponent_bullets(player1.get_bullets())
                    choice, amount ,state= ai_choice_q_table( player1.get_bullets(),player1.get_opponent_bullets())
                    if choice == 's':
                        player1.set_shoot(amount)
                        player1.set_bullets(player1.get_bullets() - amount)
                        player1.set_charged(False)
                        print(f"Player 1 pulled out a {player1.get_shoot()} and", end=" ")
                    elif choice == 'b':
                        player1.set_block(amount)
                        player1.set_shoot(0)
                        player1.set_charged(False)
                        print(f"Player 1 blocked with a {player1.get_block()} and", end=" ")
                    elif choice == 'r':
                        player1.set_bullets(player1.get_bullets() + 1)
                        player1.set_shoot(0)
                        player1.set_block(0)
                        player1.set_charged(True)
                        print("Player 1 charged and", end=" ")

                    choice, amount = ai_choice( pc.get_bullets(),player1.get_opponent_bullets())
                    if choice == 's':
                        pc.set_shoot(amount)
                        pc.set_bullets(pc.get_bullets() - amount)
                        pc.set_charged(False)
                        print(f"the CPU pulled out a {pc.get_shoot()}!")
                        pc.shooted = True
                    elif choice == 'b':
                        pc.set_block(amount)
                        pc.set_shoot(0)
                        pc.set_charged(False)
                        print(f"the CPU blocked with a {pc.get_block()}!")
                        pc.blocked = True
                    elif choice == 'r':
                        pc.set_bullets(pc.get_bullets() + 1)
                        pc.set_shoot(0)
                        pc.set_block(0)
                        pc.set_charged(True)
                        pc.set_opponent_bullets(player1.get_bullets())
                        print("the CPU charged.")

                    logic = True
                if is_death(player1.get_shoot(),pc.get_shoot(),player1.get_block(), pc.get_block()):
                    player1.set_alive(False)
                if is_death(pc.get_shoot(),player1.get_shoot(),pc.get_block(), player1.get_block()):
                    pc.set_alive(False)
            if not player1.is_alive:
                print("PC AI won!")
                pcwins = pcwins + 1
                break
            elif not pc.is_alive:
                print("Player 1 won!")
                player1wins = player1wins + 1
                break
            turn_count += 1

    print("Player 1 wins:", player1wins)
    print("PC AI wins:", pcwins)

if __name__ == "__main__":
    main()

Streaming output truncated to the last 5000 lines.
Both of them charged.

Player 1 bullets: 1
PC bullets: 1

[ 2.67620581 17.18881584  0.          0.          0.          0.
  0.          3.09686808  0.          0.          0.          0.
  0.        ]
Player 1 pulled out a 1 and the CPU blocked with a 1!
Player 1 bullets: 0
PC bullets: 1

[ 4.35875616  0.          0.          0.          0.          0.
  0.         17.72724107  0.          0.          0.          0.
  0.        ]
Player 1 blocked with a 1 and the CPU charged.
Player 1 bullets: 0
PC bullets: 2

[2.40056418 0.         0.         0.         0.         0.
 0.         1.72460975 1.78902114 0.         0.         0.
 0.        ]
Player 1 charged and the CPU pulled out a 2!
PC AI won!
Let's watch two AIs play against each other!

Turn 1!
Both of them charged.

Player 1 bullets: 1
PC bullets: 1

[ 2.67620581 17.18881584  0.          0.          0.          0.
  0.          3.09686808  0.          0.          0.          0.
  0